# Lab 04: Linear Regression and Causality
cross-sectional estimator

This lab asks you to do two tasks:

First, to estimate the effect of Right Hearth
Catheterization (RHC)--inserting a tiny hollow tube along the arterial into
the right side of your hearth.  It is often done for diagnostical
purposes (it allows to get different measures right in the hearth) and
it's usually considered safe.

Wou will use a dataset about RHC for critically ill patients and see
if RHC is related to increased death rate.  The dataset is downloaded
from [Vanderbilt
Biostats](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) and
more information is available at
[http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html).

Second, you have to create (or re-create) the design matrices of the
first problem, analyze their properties, and compute the regression
coefficients of the first part "manually", i.e. by a simple matrix
expression on computer.

You can use both R and python.

In [1]:
# Numpy is a library for working with Arrays
import numpy as np
print("Numpy version:        %6.6s" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print("SciPy version:        %6.6s" % sp.__version__)
import scipy.stats as st

# Pandas makes working with data tables easier
import pandas as pd
print("Pandas version:       %6.6s" % pd.__version__)

# Module for plotting
import matplotlib 
print("Maplotlib version:    %6.6s" % matplotlib.__version__)
%matplotlib inline
# needed for inline plots in notebooks
import matplotlib.pyplot as plt  

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print("Scikit-Learn version: %6.6s" % sklearn.__version__)

import statsmodels.api as sm

Numpy version:        1.13.3
SciPy version:        0.19.1
Pandas version:       0.20.3
Maplotlib version:     2.1.0
Scikit-Learn version: 0.19.1


/Users/kathryn/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## 1 Get Ready

Load the data.  A tab-separated version is available on canvas (files/data).

1. How many observation and variables do we have?
2. The most important variables we use below are _death_ (patient
death recorded/not recorded) and
_swang1_ (rhc performed/not performed).  How are these variables coded?

In [2]:
data = pd.read_csv("hearth_catheterization.csv", sep='\t')
print(data)

                   cat1               cat2          ca  sadmdte  dschdte  \
0                  COPD                NaN         Yes    11142  11151.0   
1         MOSF w/Sepsis                NaN          No    11799  11844.0   
2     MOSF w/Malignancy      MOSF w/Sepsis         Yes    12083  12143.0   
3                   ARF                NaN          No    11146  11183.0   
4         MOSF w/Sepsis                NaN          No    12035  12037.0   
5                  COPD                NaN          No    12389  12396.0   
6     MOSF w/Malignancy                NaN  Metastatic    12381  12423.0   
7                   ARF               Coma          No    11453  11487.0   
8     MOSF w/Malignancy                NaN         Yes    12426  12437.0   
9                   ARF                NaN         Yes    11381  11400.0   
10                  ARF                NaN          No    11705  11724.0   
11                  ARF                NaN          No    12174  12187.0   
12        MO

In [3]:
print('keys: ', data.keys())
print('# of variables: ', len(data.keys()))
print('# of observations: ', data.shape[0])

keys:  Index(['cat1', 'cat2', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte',
       'death', 'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx',
       'renalhx', 'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx',
       'amihx', 'age', 'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30',
       'aps1', 'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1',
       'alb1', 'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1',
       'swang1', 'wtkilo1', 'dnr1', 'ninsclas', 'resp', 'card', 'neuro',
       'gastr', 'renal', 'meta', 'hema', 'seps', 'trauma', 'ortho', 'adld3p',
       'urin1', 'race', 'income', 'ptid'],
      dtype='object')
# of variables:  62
# of observations:  5735


## 2 Cross-Sectional Estimator

Estimate, using linear regression, how is the RHC related to mortality.

We start easy: let's just find the relationship between recorded death
and rhc.  The important variables are

* _death_: patient death recorded/not recorded
* _swang1_: rhc performed/not performed

This is called "cross-sectional estimator", we just compare
cross-section of individuals who received and did not receive RHC.

Obviously, this is a very crude measure because the hospitals track
patients over different time periods, and if contact is lost early,
the death may not be recorded.  Another obvious problem is that the
patients have very different medical conditions, a factor that
most likely plays a role in the decision whether to perform RHC.


In [4]:
# convert to booleans
data['death'] = (data['death'] == 'Yes').astype(int)
data['swang1'] = (data['swang1'] == 'RHC').astype(int)

In [5]:
x = data['swang1'].as_matrix() # x = rhc
y = data['death'].as_matrix() # y = death

# First Method
slope, intercept, r_value, p_value, std_err = st.linregress(x, y)
print('Std. error: ', std_err) # Standard error of the estimated gradient
print('p value: ', p_value) # two-sided p-value for a hypothesis test whose null hypothesis is that the slope is zero
print('r value (correlation): ', r_value) # correlation coefficient
print('Slope (b1): ', slope) # slope of the regression line
print('Intercept: (b0): ', intercept) # intercept of the regression line

# Second Method
y = data['death']
x = data['swang1']
model = sm.OLS(y, sm.add_constant(x)).fit()
model.summary()

Std. error:  0.0129639822118
p value:  9.24152003235e-05
r value (correlation):  0.0516036805861
Slope (b1):  0.0507211506226
Intercept: (b0):  0.62968177978


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  death   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     15.31
Date:                Wed, 31 Jan 2018   Prob (F-statistic):           9.24e-05
Time:                        23:22:07   Log-Likelihood:                -3888.1
No. Observations:                5735   AIC:                             7780.
Df Residuals:                    5733   BIC:                             7794.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6297      0.008     78.709      0.000       0.614       0.645
swang1         0.0507      0.013      3.912      0.000       0.025       0.076
==============================================================================
Omnibus:                      315.261   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              982.605
Skew:                          -0.622   Prob(JB):                    4.27e-214
Kurtosis:                       1.398   Cond. No.                         2.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Comment your results.

RHC seems to be associated with 5% larger probability of death.  The
association is highly significant (t=3.9)

In [6]:
# We compare the p-value to alpha, which we will let alpha be 0.05.
# Since the p value (9.24152003235e-05) is less than 0.05, we reject the null hypothesis
# that there is no significant correlation in favor of the alternative hypothesis.
# Therefore, the correlation between death and RHC is significant.
# Because slope is positive (0.0507211506226), the relationship is positive. Meaning that
# having RHC corresponds with more death.

## 3 Address some of the issues

Now let's try to address some of the issues with the estimator above.
The dataset includes many other patient descriptors.


### 3.1 Let's include 'age' and 'sex'.
How are these coded?

In [7]:
data['age'] # Floats: e.g. 70.250977
data['sex'] # Strings: "Male", "Female"

0         Male
1       Female
2       Female
3       Female
4         Male
5       Female
6         Male
7         Male
8       Female
9       Female
10        Male
11        Male
12        Male
13      Female
14        Male
15        Male
16        Male
17      Female
18        Male
19        Male
20        Male
21      Female
22        Male
23        Male
24        Male
25      Female
26        Male
27      Female
28        Male
29        Male
         ...  
5705    Female
5706      Male
5707      Male
5708      Male
5709    Female
5710    Female
5711    Female
5712      Male
5713    Female
5714      Male
5715    Female
5716      Male
5717      Male
5718    Female
5719      Male
5720    Female
5721      Male
5722    Female
5723      Male
5724    Female
5725    Female
5726    Female
5727      Male
5728      Male
5729      Male
5730      Male
5731    Female
5732      Male
5733      Male
5734    Female
Name: sex, Length: 5735, dtype: object

What do you find?

We see age is numeric and sex is 'Female'/'Male'.

### 3.2 Include age and sex in the regression

Now allow the death to depend on gender and age, on top of that it may
depend on rhc.  Note that it may not just depend on age in a linear
fashion but in a much more complex way, so include not just $age$ but
also $age^2$ and $age^3$ as explanatory variables.

In [8]:
# convert sex to booleans
data['sex'] = (data['sex'] == 'Male').astype(int)

In [9]:
# does death depend on sex and age?
y = data['death']

# explanatory variables
data = data.assign(age2 = data['age'] ** 2)
data = data.assign(age3 = data['age'] ** 3)
x = data[['age', 'age2', 'age3','sex', 'swang1']]
model = sm.OLS(y, sm.add_constant(x)).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  death   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     60.92
Date:                Wed, 31 Jan 2018   Prob (F-statistic):           4.72e-62
Time:                        23:22:10   Log-Likelihood:                -3747.2
No. Observations:                5735   AIC:                             7506.
Df Residuals:                    5729   BIC:                             7546.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1185      0.151      0.786      0.432      -0.177       0.414
age            0.0117      0.009      1.324      0.186      -0.006       0.029
age2       -7.451e-05      0.000     -0.463      0.644      -0.000       0.000
age3        2.964e-07   9.33e-07      0.318      0.751   -1.53e-06    2.12e-06
sex            0.0230      0.012      1.854      0.064      -0.001       0.047
swang1         0.0546      0.013      4.291      0.000       0.030       0.080
==============================================================================
Omnibus:                      288.479   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              818.571
Skew:                          -0.591   Prob(JB):                    1.78e-178
Kurtosis:                       1.575   Cond. No.                     8.26e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.26e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Comment your results

Seems like the association between treatment and rhc is not much
affect, if anything, it got larger.

In [10]:
# t is 4.291, which is larger than previous association of just death and rhc

## 4 Design matrices of the models

Each linear model (and many other models) have associated _design
matrices_.  Design matrix is the matrix of all your explanatory
variables (all x-s) in the final numeric matrix form.  This includes
* adding a constant column
* converting categorical/non-numeric variables into suitable numeric
variables

You next task is to create/extract the design matrices of both of the
models above, investigate their numeric properties (condition
numbers), and solve the linear regression problem in matrix form.

We did not have time in the class to talk about it, but there is a
closed-form solution for the linear regression problem:  beta =
$(X'X)^{-1} X'y$.  Compute this solution and compare with the regression
package output above.


### 4.1 First model

#### 4.1.1 create the design matrix of it, X.

Depending on the way you solved your problem, you may already have
created it.  Depending on the way you solved the problem above, you
may be able to extract it from the existing model.  You may also redo it
manually here.  Remember:
* include the constant term!
* design matrix must be a _matrix_, not data frame or something else.

In [17]:
d = data[['swang1']]
X = sm.add_constant(d).as_matrix()
X

array([[ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  1.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

#### 4.1.2 Compute the condition number of X`X.

You may choose whatever definition you like, but please report what
are you using.

In [18]:
XX = np.matmul(X.T, X)

condition = np.linalg.cond(XX)
print('condition #: ', condition)# 5.91706511753

condition #:  5.91706511753


#### 4.1.3 Compute your regression coefficients using the formula above.

Note: you also need your outcome variable $y$ in numeric matrix
form. 

In [28]:
y = data[['death']].as_matrix()
beta = np.matmul(np.linalg.inv(XX),X.T)
beta = np.matmul(beta, y)
beta # column vector

array([[  1.18546603e-01],
       [  1.16526175e-02],
       [ -7.45104137e-05],
       [  2.96365651e-07],
       [  2.30153708e-02],
       [  5.46474537e-02]])

#### 4.1.4 Compare your coefficients here with the OLS results above.

In [ ]:
# They are the same.

### 4.2 Second Model

Repeat the previous steps 1..4 for the second model, involving $age$,
$age^2$, and $age^3$.

In [19]:
d = data[['age', 'age2', 'age3','sex', 'swang1']]
X = sm.add_constant(d).as_matrix()
X

array([[  1.00000000e+00,   7.02509766e+01,   4.93519971e+03,
          3.46702600e+05,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   7.81789551e+01,   6.11194902e+03,
          4.77825788e+05,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   4.60919800e+01,   2.12447062e+03,
          9.79210573e+04,   0.00000000e+00,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   8.04849854e+01,   6.47783287e+03,
          5.21368284e+05,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   6.73789673e+01,   4.53992523e+03,
          3.05895474e+05,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   5.46639709e+01,   2.98814971e+03,
          1.63344129e+05,   0.00000000e+00,   0.00000000e+00]])

In [20]:
XX = np.matmul(X.T, X)

condition = np.linalg.cond(XX)
print('condition #: ', condition) # 6.81925770479e+13

condition #:  6.81925770479e+13


Compare the results with the regression output.  Comment the condition
numbers you got.

The coefficients are the same.  One can see that the condition number
is enormous, so there may be numeric problems.
